In [31]:
import torch
from botorch.models import SingleTaskGP
from botorch.fit import fit_gpytorch_model
from botorch.utils import standardize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.models.gpytorch import GPyTorchModel
from gpytorch.distributions import MultivariateNormal
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP
from gpytorch.kernels import RBFKernel, ScaleKernel
from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.priors import GammaPrior

In [32]:
train_x = torch.Tensor([0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02,0.01, 0.009, 0.008, 0.007, 0.006, 0.005,0.0051, 0.004, 0.003, 0.002, 0.001, 0.0009, 0.0008, ])
train_y = torch.Tensor([9.34, 28.98, 39.05, 9.36, 9.41, 69.7, 53.63, 129.48, 181.43, 198.67, 196.13, 195.13, 196.3, 196.73, 197.87,198, 196.87, 198, 197.63, 197.33, 197.4, 197.33])

In [33]:
#train_y = torch.Tensor([100,110,120,130,140,150,160,170,180,190,200,190,180,170,160,150,140,130,120,110,100])

In [34]:
print(len(train_y))

22


In [35]:

class SimpleCustomGP(ExactGP, GPyTorchModel):

    _num_outputs = 1  # to inform GPyTorchModel API
    
    def __init__(self, train_X, train_Y):
        # squeeze output dim before passing train_Y to ExactGP
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(
            base_kernel=RBFKernel(ard_num_dims=train_X.shape[-1]),
        )
        self.to(train_X)  # make sure we're on the right device/dtype
        
    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [36]:
gp = SimpleCustomGP(train_x,train_y)
mll = ExactMarginalLogLikelihood(gp.likelihood, gp)
fit_gpytorch_model(mll)

ExactMarginalLogLikelihood(
  (likelihood): GaussianLikelihood(
    (noise_covar): HomoskedasticNoise(
      (raw_noise_constraint): GreaterThan(1.000E-04)
    )
  )
  (model): SimpleCustomGP(
    (likelihood): GaussianLikelihood(
      (noise_covar): HomoskedasticNoise(
        (raw_noise_constraint): GreaterThan(1.000E-04)
      )
    )
    (mean_module): ConstantMean()
    (covar_module): ScaleKernel(
      (base_kernel): RBFKernel(
        (raw_lengthscale_constraint): Positive()
        (distance_module): Distance()
      )
      (raw_outputscale_constraint): Positive()
    )
  )
)

In [192]:
from botorch.acquisition import UpperConfidenceBound
UCB = UpperConfidenceBound(gp,beta=0.01)

In [193]:
bounds = torch.stack([torch.zeros(1), torch.ones(1)])
print(bounds)

tensor([[0.],
        [1.]])


In [200]:
from botorch.optim import optimize_acqf
bounds = torch.stack([torch.zeros(1), torch.ones(1)])
candidate, acq_value = optimize_acqf(UCB, bounds=bounds, q=1, num_restarts=5, raw_samples=20)
print(candidate)

tensor([[0.0052]])


In [40]:
print(candidate)

tensor([[0.0052]])
